In [1]:
import MARBL_tools # Note: I softlinked $MARBL/MARBL_tools into the defaults directory
                   #       I also needed to update some of the python to grab more data from the JSON file
    
import yaml
import pandas as pd

%load_ext watermark
%watermark -d -iv -m -g -h

pandas 1.0.3
yaml   5.3.1
2020-05-08 

compiler   : GCC 7.3.0
system     : Linux
release    : 3.10.0-693.21.1.el7.x86_64
machine    : x86_64
processor  : x86_64
CPU cores  : 72
interpreter: 64bit
host name  : casper21
Git hash   : 3a69d4ecbc39b859f2c5f30fab965453a31b0ce8


In [2]:
with open('MARBL_settings.yaml', encoding='utf-8') as file_in:
    config_dict = yaml.safe_load(file_in)
MARBL_settings = MARBL_tools.MARBL_settings_class('defaults/json/settings_cesm2.1.json',
                                                  config_dict['defaults']['saved_state_source'],
                                                  config_dict['defaults']['grid']
                                                 )

In [3]:
# I think the easiest way to have consistency in units is to create a function here
def _unit_conversion(old_units):
    new_units = None
    
    if old_units == "1/day":
        new_units = "$\mathrm{d^{-1}}$"
    if new_units is not None:
        return new_units
    return old_units

In [4]:
table_dict = {}
table_dict['Parameter'] = []
table_dict['Value'] = []
table_dict['Units'] = []
table_dict['Description'] = []
for varname in config_dict['Parameters']:
    # Parameters column contains variable name unless a different label was specified
    try:
        table_dict['Parameter'].append(f"{config_dict['Parameters'][varname]['label']}")
    except:
        table_dict['Parameter'].append(varname)

    # Need a way to standardize units
    table_dict['Units'].append(_unit_conversion(MARBL_settings.settings_dict[varname]['units']))

    # Description column contains variable longname unless a different description was specified
    try:
        table_dict['Description'].append(config_dict['Parameters'][varname]['description'])
    except:
        table_dict['Description'].append(MARBL_settings.settings_dict[varname]['longname'])
    # value comes from parsing the JSON file
    table_dict['Value'].append(MARBL_settings.settings_dict[varname]['value'])


In [5]:
# Make table
pd.DataFrame(table_dict)

,Parameter,Value,Units,Description
0,"$\mu_{\mathrm{ref},\mathrm{sp}}$",5.0,$\mathrm{d^{-1}}$,Maximum C-spec growth rate at Tref
1,"$\mu_{\mathrm{ref},\mathrm{diat}}$",5.0,$\mathrm{d^{-1}}$,Maximum C-spec growth rate at Tref
2,"$\mu_{\mathrm{ref},\mathrm{diaz}}$",2.5,$\mathrm{d^{-1}}$,Maximum C-spec growth rate at Tref


In [6]:
print(pd.DataFrame(table_dict).to_latex(index=False, escape=False))

\begin{tabular}{lrll}
\toprule
                          Parameter &  Value &              Units &                         Description \\
\midrule
   $\mu_{\mathrm{ref},\mathrm{sp}}$ &    5.0 &  $\mathrm{d^{-1}}$ &  Maximum C-spec growth rate at Tref \\
 $\mu_{\mathrm{ref},\mathrm{diat}}$ &    5.0 &  $\mathrm{d^{-1}}$ &  Maximum C-spec growth rate at Tref \\
 $\mu_{\mathrm{ref},\mathrm{diaz}}$ &    2.5 &  $\mathrm{d^{-1}}$ &  Maximum C-spec growth rate at Tref \\
\bottomrule
\end{tabular}

